In [1]:
import pandas as pd 

In [2]:
aisles = pd.read_csv("./src/aisles.csv")
aisles

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [3]:
departments = pd.read_csv("./src/departments.csv")
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [4]:
#order_products__prior = pd.read_csv("./src/order_products__prior.csv")
#order_products__prior

In [5]:
order_products__train = pd.read_csv("./src/order_products__train.csv")
order_products__train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [6]:
orders = pd.read_csv("./src/orders.csv").fillna(value = -1)   
#make it -1 if empty, and the prediction would be 0!
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,-1.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [7]:
#change order_dow to weekend
#1 means weekend, 0 means weekday
#0 is Saturday, 1 is Sunday ...

is_weekend = []
for dow in orders['order_dow']:
    if dow <= 1:
        is_weekend.append(1)
    else:
        is_weekend.append(0)
orders = orders.drop(columns=['order_dow'])
orders['is_weekend'] = pd.DataFrame(is_weekend)
orders

NameError: name 'weekend' is not defined

In [62]:
#change order_hour_of_day to daytime
#1 means daytime, 0 means nighttime
is_daytime = []
for hour in orders['order_hour_of_day']:
    if hour < 7 or hour > 19:
        is_daytime.append(0)
    else:
        is_daytime.append(1)
orders = orders.drop(columns=['order_hour_of_day'])
orders['is_daytime'] = pd.DataFrame(is_daytime)
orders

,order_id,user_id,eval_set,order_number,days_since_prior_order,is_weekend,is_daytime
0,2539329,1,prior,1,-1.0,0,1
1,2398795,1,prior,2,15.0,0,1
2,473747,1,prior,3,21.0,0,1
3,2254736,1,prior,4,29.0,0,1
4,431534,1,prior,5,28.0,0,1
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,29.0,0,1
3421079,1854736,206209,prior,11,30.0,0,1
3421080,626363,206209,prior,12,18.0,1,1
3421081,2977660,206209,prior,13,7.0,1,1


In [8]:
products = pd.read_csv("./src/products.csv")
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [51]:
products = pd.get_dummies(products, prefix=['department_id_is'], columns=['department_id'])

In [52]:
def merge_dfs(order_products, orders, products):
    df = order_products
    df = pd.merge(df, orders, how = 'left', on = 'order_id')
    df = pd.merge(df, products.drop(columns=['product_name']), how = 'left', on = 'product_id')
    return df

In [65]:
train_df = merge_dfs(order_products__train, orders, products)
train_df

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,...,department_id_is_12,department_id_is_13,department_id_is_14,department_id_is_15,department_id_is_16,department_id_is_17,department_id_is_18,department_id_is_19,department_id_is_20,department_id_is_21
0,1,49302,1,1,112108,train,4,4,10,9.0,...,0,0,0,0,1,0,0,0,0,0
1,1,11109,2,1,112108,train,4,4,10,9.0,...,0,0,0,0,1,0,0,0,0,0
2,1,10246,3,0,112108,train,4,4,10,9.0,...,0,0,0,0,0,0,0,0,0,0
3,1,49683,4,0,112108,train,4,4,10,9.0,...,0,0,0,0,0,0,0,0,0,0
4,1,43633,5,1,112108,train,4,4,10,9.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,169679,train,30,0,10,4.0,...,0,0,0,0,0,0,0,0,0,0
1384613,3421063,35548,4,1,169679,train,30,0,10,4.0,...,0,0,0,0,0,0,0,0,1,0
1384614,3421070,35951,1,1,139822,train,15,6,10,8.0,...,0,0,0,0,1,0,0,0,0,0
1384615,3421070,16953,2,1,139822,train,15,6,10,8.0,...,0,1,0,0,0,0,0,0,0,0


In [66]:
train_info = pd.DataFrame(train_df, columns=['order_id', 'product_id'])
train_info

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633
...,...,...
1384612,3421063,14233
1384613,3421063,35548
1384614,3421070,35951
1384615,3421070,16953


In [67]:
train_df = train_df.drop(columns = ['order_id', 'product_id', 'user_id', 'eval_set', 'aisle_id'])
train_df

,add_to_cart_order,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order,department_id_is_1,department_id_is_2,department_id_is_3,department_id_is_4,...,department_id_is_12,department_id_is_13,department_id_is_14,department_id_is_15,department_id_is_16,department_id_is_17,department_id_is_18,department_id_is_19,department_id_is_20,department_id_is_21
0,1,1,4,4,10,9.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,1,4,4,10,9.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,0,4,4,10,9.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,4,4,10,9.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,1,4,4,10,9.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3,1,30,0,10,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1384613,4,1,30,0,10,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1384614,1,1,15,6,10,8.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1384615,2,1,15,6,10,8.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [68]:
train_y = pd.DataFrame(train_df, columns = ['reordered'])
train_y

,reordered
0,1
1,1
2,0
3,0
4,1
...,...
1384612,1
1384613,1
1384614,1
1384615,1


In [69]:
train_x = train_df.drop(columns = ['reordered'])
train_x

,add_to_cart_order,order_number,order_dow,order_hour_of_day,days_since_prior_order,department_id_is_1,department_id_is_2,department_id_is_3,department_id_is_4,department_id_is_5,...,department_id_is_12,department_id_is_13,department_id_is_14,department_id_is_15,department_id_is_16,department_id_is_17,department_id_is_18,department_id_is_19,department_id_is_20,department_id_is_21
0,1,4,4,10,9.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,4,4,10,9.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,4,4,10,9.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,4,4,10,9.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,4,4,10,9.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3,30,0,10,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1384613,4,30,0,10,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1384614,1,15,6,10,8.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1384615,2,15,6,10,8.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [70]:
train_info.to_csv("train_info.csv", index = False)
train_x.to_csv("train_x.csv", index = False)
train_y.to_csv("train_y.csv", index = False)

In [ ]:
# Right now, we have only considered if an order has a prior order
# We haven't considered whether a product has been re-ordered
# If we are going to take the 2nd one into consideration, 
# we need to calculate : 1) how many times it has been ordered before;    2) how many days since the last time it was ordered
# However, this approach would be too complex, since it must be customed for each user